<a href="https://colab.research.google.com/github/stinkymatt/pub/blob/master/AstraGeohashSamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-secret-manager
!pip install astrapy
!pip install cassandra-driver

In [ ]:
#Credentials for a Read-only user
auth_id = "uqGCqRUUZpbzzJYHOpvcziCN" #@param {type: "string"}
auth_secret = "iOwv-AENt2xNK67Sug8UmS.j4Jp6DOZZ6IPWkGxIKivINqRbER+n-OxGsF_9a5Hl2GsygGrc1NEoPZo7Poi._f_bWHqFPA4eQXJKyDWTMJ5-aUl.K_xUU20kcy-9Z.LY" #@param {type:"string"}
auth_token = "AstraCS:uqGCqRUUZpbzzJYHOpvcziCN:02389ffc8a18d706e99ec43f0c9c66bde3368a4b2da8a3cc56ca4f109692c0e5" #@param {type: "string"}

ASTRA_DB_ID='c2902cac-68fa-4d5a-91dd-bc8163619c98'
ASTRA_DB_REGION='us-east4'
ASTRA_DB_KEYSPACE='geodemos'

from astrapy.client import create_astra_client
astra_client = create_astra_client(astra_database_id=ASTRA_DB_ID,
                                   astra_database_region=ASTRA_DB_REGION,
                                   astra_application_token=auth_token)
zipUrl = astra_client.ops.get_secure_bundle(database=ASTRA_DB_ID)

import urllib
urllib.request.urlretrieve(zipUrl['downloadURL'], 'scb.zip')

('scb.zip', <http.client.HTTPMessage at 0x7f6b64361690>)

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

def connect_to_astra():
  cloud_config= {'secure_connect_bundle':'scb.zip'}
  auth_prov = PlainTextAuthProvider(auth_id, auth_secret)
  cluster = Cluster(cloud=cloud_config, auth_provider=auth_prov)
  return cluster.connect()

#Connect to Astra and query from geodemos.my_mapdata table
#There are string matching indexes on geo_hash3, geo_hash4, geo_hash5 and geo_hash10
#The general idea is that you could move easily between the 3/4/5 "zoom levels"
#9xj3t is a crowded 5-character geo_hash to try out. 
#There are 250,000 items in this DB
session = connect_to_astra()

rows = session.execute("select md_pk, md_name, md_lat, md_lng, geo_hash10 from geodemos.my_mapdata where geo_hash5='9xj3t'")
#rows = session.execute("select md_pk, md_name, md_lat, md_lng, geo_hash10 from geodemos.my_mapdata where geo_hash4='9xj3'")
#rows = session.execute("select md_pk, md_name, md_lat, md_lng, geo_hash10 from geodemos.my_mapdata where geo_hash3='9xj'")
rows.all()

#Just for pretty printing below
#import pandas as pd
#from google.colab import data_table
#data_table.enable_dataframe_formatter()
#data = pd.DataFrame(rows) 
#data


[Row(md_pk='SG_5CdrDpKuqyYo5c1dRs2ENv_39.699799_-104.937234@1303263093', md_name='Nail Studio 132', md_lat='39.699799', md_lng='-104.937234', geo_hash10='9xj3u55h14'),
 Row(md_pk='SG_3AscSaVIlgSmodPwr4jkgG_39.666767_-104.988904@1294083659', md_name='Lorax Landscaping LLC', md_lat='39.666767', md_lng='-104.988904', geo_hash10='9xj3dvj7tj'),
 Row(md_pk='SG_59Y3ADEvJxNvkw3h2RYA25_39.657599_-105.000008@1294083659', md_name='Dash & Azua Cabinet Makers', md_lat='39.657599', md_lng='-105.000008', geo_hash10='9xj3demq62'),
 Row(md_pk='SG_6PNvOglIDKyIHS3IO5jCqb_39.917789_-105.039138@1294083659', md_name='Greg Mackey Oil & Operating', md_lat='39.917789', md_lng='-105.039138', geo_hash10='9xj71fbcfx'),
 Row(md_pk='SG_7eV1BGiaq7HCZ32iyynivn_39.540293_-105.296475@1294083659', md_name='Security Title Guaranty Co', md_lat='39.540293', md_lng='-105.296475', geo_hash10='9xj0gyj6yk'),
 Row(md_pk='SG_0kRHyVECidOjiLwmnSulfV_39.795595_-104.906571@1294083659', md_name='Kenyon Construction Inc', md_lat='39.7